In [ ]:
import numpy as np
import pandas as pd
from blpapi import Session, Event
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from sklearn.preprocessing import MinMaxScaler

class PutOptionOptimizer:
    def __init__(self, ticker='SPX Index', start_date='20100101', end_date=None):
        self.ticker = ticker
        self.data = self._load_bloomberg_data(start_date, end_date)
        self.results = None

    def _load_bloomberg_data(self, start_date, end_date):
        """Load SPX historical data with volatility"""
        try:
            session = Session()
            if not session.start():
                raise ConnectionError("Failed to start Bloomberg session")
            
            session.openService("//blp/refdata")
            service = session.getService("//blp/refdata")
            request = service.createRequest("HistoricalDataRequest")
            request.getElement("securities").appendValue(self.ticker)
            request.getElement("fields").appendValue("PX_LAST")
            request.getElement("fields").appendValue("DAILY_VOL")
            request.set("periodicityAdjustment", "ACTUAL")
            request.set("periodicitySelection", "DAILY")
            request.set("startDate", start_date)
            request.set("endDate", end_date or datetime.today().strftime('%Y%m%d'))
            
            session.sendRequest(request)
            data = []
            while True:
                event = session.nextEvent()
                if event.eventType() == Event.PARTIAL_RESPONSE:
                    self._process_response(event, data)
                elif event.eventType() == Event.RESPONSE:
                    self._process_response(event, data)
                    break
            
            df = pd.DataFrame(data)
            df['date'] = pd.to_datetime(df['date'])
            df = df.set_index('date').rename(columns={'PX_LAST':'price', 'DAILY_VOL':'daily_vol'})
            df['returns'] = np.log(df['price']/df['price'].shift(1))
            df['volatility'] = df['returns'].rolling(21).std() * np.sqrt(252)
            return df.dropna()
            
        finally:
            session.stop()

    def _process_response(self, event, data_container):
        """Process Bloomberg response with volatility data"""
        for msg in event:
            field_data = msg.getElement("securityData").getElement("fieldData")
            for i in range(field_data.numValues()):
                elem = field_data.getValueAsElement(i)
                date = elem.getElementAsDatetime("date").strftime('%Y-%m-%d')
                price = elem.getElementAsFloat("PX_LAST")
                vol = elem.getElementAsFloat("DAILY_VOL") if elem.hasElement("DAILY_VOL") else np.nan
                data_container.append({'date':date, 'PX_LAST':price, 'DAILY_VOL':vol})

    def _estimate_option_pnl(self, entry_date, strike_pct, maturity_days):
        """
        Estimate daily PnL path for a put option using Black-Scholes delta approximation
        Returns: DataFrame with daily MTM values
        """
        entry_idx = self.data.index.get_loc(entry_date)
        expiry_idx = min(entry_idx + maturity_days, len(self.data)-1)
        
        spot_prices = self.data.iloc[entry_idx:expiry_idx+1]['price']
        volatilities = self.data.iloc[entry_idx:expiry_idx+1]['volatility'].fillna(0.2)
        days_to_expiry = np.arange(maturity_days, -1, -1)[:len(spot_prices)]
        
        strike_price = spot_prices.iloc[0] * strike_pct
        intrinsic_values = np.maximum(strike_price - spot_prices, 0)
        
        # Simplified MTM estimation using delta approximation
        moneyness = np.log(strike_price/spot_prices)
        time_decay = days_to_expiry/365
        d1 = (moneyness + 0.5*volatilities**2*time_decay) / (volatilities*np.sqrt(time_decay))
        deltas = -norm.cdf(-d1)  # Put option delta
        
        # MTM = Intrinsic + Time Value approximation
        mtm_values = intrinsic_values + (spot_prices * np.abs(deltas) * 0.1)  # Time value heuristic
        
        return pd.DataFrame({
            'date': spot_prices.index,
            'spot_price': spot_prices.values,
            'mtm': mtm_values.values,
            'daily_pnl': mtm_values.diff().values,
            'cumulative_pnl': mtm_values - mtm_values.iloc[0]
        })

    def evaluate_strategy(self, params):
        """
        Evaluate strategy with blended objective:
        - 50% weight to intrinsic value at expiry
        - 30% weight to cumulative PnL
        - 20% weight to Sharpe ratio of daily PnL
        """
        lookback, vol_threshold, z_threshold, strike_pct, maturity_days = params
        lookback = int(max(21, min(252, lookback)))
        vol_threshold = max(0.1, min(0.5, vol_threshold))
        strike_pct = max(0.7, min(0.95, strike_pct))
        maturity_days = int(max(30, min(180, maturity_days)))
        
        # Generate signals
        self.data['signal'] = (
            (self.data['volatility'].rolling(lookback).mean() > vol_threshold) &
            (zscore(self.data['price'].pct_change().rolling(lookback).mean()) > z_threshold)
        ).astype(int)
        
        all_pnls = []
        intrinsic_values = []
        
        for entry_date in self.data[self.data['signal']==1].index:
            try:
                mtm_df = self._estimate_option_pnl(entry_date, strike_pct, maturity_days)
                all_pnls.append(mtm_df['daily_pnl'].dropna())
                intrinsic_values.append(mtm_df['mtm'].iloc[-1])  # Intrinsic at expiry
            except:
                continue
        
        if not all_pnls:
            return -np.inf  # Minimize in optimization
        
        # Combine metrics
        total_intrinsic = np.sum(intrinsic_values)
        avg_pnl = np.mean([pnl.sum() for pnl in all_pnls])
        sharpe = np.mean([pnl.mean()/pnl.std() for pnl in all_pnls if len(pnl)>1 and pnl.std()>0])
        
        # Normalized blended score
        scaler = MinMaxScaler()
        scaled_metrics = scaler.fit_transform(np.array([[total_intrinsic, avg_pnl, sharpe]]))
        blended_score = 0.5*scaled_metrics[0,0] + 0.3*scaled_metrics[0,1] + 0.2*scaled_metrics[0,2]
        
        return blended_score

    def optimize_strategy(self):
        """Optimize parameters using a blended objective function"""
        def objective(params):
            return -self.evaluate_strategy(params)  # Minimize negative score
        
        initial_guess = [63, 0.25, 1.5, 0.85, 90]  # lookback, vol_thresh, z_thresh, strike, maturity
        bounds = [
            (21, 252),    # Lookback window
            (0.15, 0.4),  # Volatility threshold
            (1.0, 2.5),   # Z-score threshold
            (0.75, 0.9),  # Strike percentage
            (30, 180)     # Maturity days
        ]
        
        result = minimize(objective, initial_guess, bounds=bounds, method='L-BFGS-B')
        optimal_params = result.x
        optimal_score = -result.fun
        
        # Store best parameters
        self.results = {
            'lookback_days': int(optimal_params[0]),
            'volatility_threshold': optimal_params[1],
            'z_threshold': optimal_params[2],
            'strike_pct': optimal_params[3],
            'maturity_days': int(optimal_params[4]),
            'optimized_score': optimal_score
        }
        
        return self.results

    def backtest_strategy(self, params=None):
        """Backtest with optimized or custom parameters"""
        params = params or self.results
        if not params:
            raise ValueError("Run optimize_strategy() first or provide parameters")
        
        lookback = params['lookback_days']
        vol_thresh = params['volatility_threshold']
        z_thresh = params['z_threshold']
        strike_pct = params['strike_pct']
        maturity = params['maturity_days']
        
        # Generate signals
        self.data['signal'] = (
            (self.data['volatility'].rolling(lookback).mean() > vol_thresh) &
            (zscore(self.data['price'].pct_change().rolling(lookback).mean()) > z_thresh)
        ).astype(int)
        
        all_trades = []
        for entry_date in self.data[self.data['signal']==1].index:
            try:
                mtm_df = self._estimate_option_pnl(entry_date, strike_pct, maturity)
                trade_result = {
                    'entry_date': entry_date,
                    'exit_date': mtm_df['date'].iloc[-1],
                    'strike_price': mtm_df['spot_price'].iloc[0] * strike_pct,
                    'max_pnl': mtm_df['cumulative_pnl'].max(),
                    'final_pnl': mtm_df['cumulative_pnl'].iloc[-1],
                    'avg_daily_pnl': mtm_df['daily_pnl'].mean(),
                    'sharpe': mtm_df['daily_pnl'].mean() / mtm_df['daily_pnl'].std()
                }
                all_trades.append(trade_result)
            except:
                continue
        
        return pd.DataFrame(all_trades)

# Example Usage
if __name__ == "__main__":
    optimizer = PutOptionOptimizer(start_date='20150101')
    
    # Step 1: Optimize parameters
    print("Optimizing strategy...")
    best_params = optimizer.optimize_strategy()
    print("\nOptimized Parameters:")
    for k, v in best_params.items():
        print(f"{k:>25}: {v:.2f}" if isinstance(v, float) else f"{k:>25}: {v}")
    
    # Step 2: Backtest
    print("\nRunning backtest...")
    backtest_results = optimizer.backtest_strategy()
    print("\nBacktest Summary:")
    print(backtest_results.describe().to_string())
    
    # Plot cumulative PnL
    plt.figure(figsize=(12,6))
    plt.plot(backtest_results['entry_date'], backtest_results['final_pnl'].cumsum())
    plt.title("Cumulative PnL of Optimized Put Strategy")
    plt.xlabel("Trade Date")
    plt.ylabel("Cumulative PnL")
    plt.grid()
    plt.show()